<a href="https://colab.research.google.com/github/MinhHieu-Tapcode/BTL_HQT_QLY_TMDT/blob/main/Code_gi%E1%BA%A3_l%E1%BA%ADp_HQT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install faker unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 9.2 MB/s eta 0:00:00


In [7]:
import random
import os
from datetime import datetime, timedelta
from unidecode import unidecode

# Try to use faker-vietnam if available (gives best VN names), else fallback
try:
    from faker_vietnam import FakerVietnam
    fake_vn = FakerVietnam()
    USE_FAKER_VN = True
except Exception:
    USE_FAKER_VN = False
    # If faker-vietnam not installed, try to use faker for some utilities
    try:
        from faker import Faker
        fake = Faker("vi_VN")
    except Exception:
        fake = None

# ---------- Helpers: nếu không có faker-vietnam, dùng danh sách VN tự tạo ----------
# Họ, tên đệm, tên phổ biến ở Việt Nam
ho_list = ["Nguyễn", "Trần", "Lê", "Phạm", "Hoàng", "Huỳnh", "Phan", "Vũ", "Võ", "Đặng", "Bùi", "Đỗ", "Ngô", "Dương"]
ten_dem_list = ["Văn", "Hữu", "Đức", "Minh", "Ngọc", "Thị", "Thu", "Thùy", "Quang", "Thanh", "Hải", "Anh", "Bảo", "Kim"]
ten_list = ["Hiếu", "Nhi", "Trang", "Huy", "Nam", "Hằng", "Linh", "Tú", "Hạnh", "Dương", "Tâm", "Khoa", "Khánh", "Hòa"]

def ten_viet():
    # Xác suất 50% có tên đệm
    if random.random() < 0.5:
        return f"{random.choice(ho_list)} {random.choice(ten_dem_list)} {random.choice(ten_list)}"
    else:
        return f"{random.choice(ho_list)} {random.choice(ten_list)}"

cities = [
    "Hà Nội","TP. Hồ Chí Minh","Đà Nẵng","Hải Phòng","Cần Thơ",
    "Nha Trang","Huế","Vinh","Buôn Ma Thuột","Bắc Ninh","Thái Nguyên"
]

districts = [
    "Quận Hoàn Kiếm","Quận Đống Đa","Quận Ba Đình","Quận Hai Bà Trưng",
    "Quận Thanh Xuân","Quận Cầu Giấy","Quận 1","Quận 3","Quận 7","Quận Tân Bình",
    "Quận Liên Chiểu","Quận Sơn Trà"
]

streets = [
    "Nguyễn Văn Cừ","Trần Hưng Đạo","Phan Đình Phùng","Lê Lợi",
    "Điện Biên Phủ","Lý Thường Kiệt","Hai Bà Trưng","Bà Triệu",
    "Nguyễn Thị Minh Khai","Phố Huế","Ngô Quyền","Lê Duẩn"
]

company_templates = [
    "Công ty TNHH {A}", "Công ty CP {A}", "Cửa hàng {A}", "Siêu thị {A}", "Shop {A}",
    "Chuỗi {A}", "Cơ sở {A}"
]
company_names_core = ["Minh Phát","Anh Phương","Hồng Hà","Sakura","Hải Nam","An Phát","Việt Store","VietShop","Metro"]

def fake_vietnam_name():
    # nếu có faker-vietnam, dùng nó
    if USE_FAKER_VN:
        return fake_vn.name()
    # nếu có faker vi_VN, thử dùng fake.name() (có thể hơi lạ), fallback vào list
    if fake:
        try:
            n = fake.name()
            # nếu kết quả ngắn hoặc kỳ, fallback
            if len(n.split()) >= 2:
                return n
        except Exception:
            pass
    return f"{random.choice(surnames)} {random.choice(middle_names)} {random.choice(given_names)}"

def fake_company():
    if USE_FAKER_VN:
        return fake_vn.company()
    core = random.choice(company_names_core)
    tpl = random.choice(company_templates)
    city = random.choice(cities)
    return f"{tpl.format(A=core)} - {city}"

def fake_city():
    if USE_FAKER_VN:
        return fake_vn.city()
    if fake:
        try:
            return fake.city()
        except Exception:
            pass
    return random.choice(cities)

def fake_address():
    if USE_FAKER_VN:
        return fake_vn.address().replace("\n", ", ")
    # street number + street + district + city
    num = random.randint(1, 300)
    street = random.choice(streets)
    district = random.choice(districts)
    city = random.choice(cities)
    return f"{num} {street}, {district}, {city}"

def fake_phone():
    # phổ biến: 03x, 07x, 08x, 09x, 05x
    prefixes = ['03','05','07','08','09']
    return f"{random.choice(prefixes)}{random.randint(10000000,99999999)}"

# escape single quotes for SQL
def esc(s):
    if s is None:
        return ''
    return str(s).replace("'", "''")

# ------------------ Script sinh dữ liệu ------------------
random.seed(42)

num_users = 150
num_stores = 30
num_products = 150
num_orders = 300
num_order_details = 600
num_delivery_persons = 15
num_shipping = 200
num_reviews = 200

# ensure output dir
output_file = "Qly_E_Commerce_Data.sql"
# nếu muốn lưu khác đường dẫn, thay biến output_file

with open(output_file, "w", encoding="utf-8") as f:
    f.write("-- Dữ liệu mẫu cho Oracle (tiếng Việt)\n")
    f.write("-- Bấm chạy toàn bộ script SQL này trong Oracle SQL Developer\n\n")

    # --- 1. NguoiDung ---
    f.write("-- 1. NguoiDung\n")
    users = []
    for i in range(1, num_users+1):
        name = ten_viet()
        birth_date = (datetime.now() - timedelta(days=random.randint(18*365, 60*365))).date()
        email_user = unidecode(name).lower().replace(' ', '.')
        domain = random.choice(['gmail.com', 'yahoo.com', 'outlook.com'])
        email = f"{email_user}{random.randint(1,99)}@{domain}"
        phone = fake_phone()
        users.append({'id': i, 'name': name, 'email': email})
        f.write(f"INSERT INTO NguoiDung (ma_nguoi_dung, ho_ten, email, sdt, ngay_sinh) VALUES ({i}, '{esc(name)}', '{esc(email)}', '{phone}', TO_DATE('{birth_date}','YYYY-MM-DD'));\n")
    f.write("\n")

    # --- 2. TaiKhoan ---
    f.write("-- 2. TaiKhoan\n")
    account_id = 1
    accounts = []

    for user in users:
        username = unidecode(user['name']).lower().replace(' ', '') + str(random.randint(10,99))
        f.write(f"INSERT INTO TaiKhoan (ma_tai_khoan, ma_nguoi_dung, ten_tai_khoan, mat_khau, loai_tai_khoan) VALUES ({account_id}, {user['id']}, '{esc(username)}', '123456', 'USER');\n")
        accounts.append({'id': account_id, 'user_id': user['id'], 'username': username})
        account_id += 1

    # STORE accounts
    store_accounts = []
    for i in range(1, num_stores+1):
        user_choice = random.choice(users)
        username = 'store_' + unidecode(user_choice['name']).lower().replace(' ', '') + str(random.randint(1,99))
        f.write(f"INSERT INTO TaiKhoan (ma_tai_khoan, ma_nguoi_dung, ten_tai_khoan, mat_khau, loai_tai_khoan) VALUES ({account_id}, {user_choice['id']}, '{esc(username)}', '123456', 'STORE');\n")
        store_accounts.append({'id': account_id, 'user_id': user_choice['id']})
        account_id += 1

    # ADMIN (1)
    f.write(f"INSERT INTO TaiKhoan (ma_tai_khoan, ma_nguoi_dung, ten_tai_khoan, mat_khau, loai_tai_khoan) VALUES ({account_id}, 1, 'admin', 'admin123', 'ADMIN');\n")
    f.write("\n")

    # --- 3. LoaiCuaHang ---
    f.write("-- 3. LoaiCuaHang\n")
    store_types = ['Thời trang','Điện tử','Gia dụng','Mỹ phẩm','Sách','Đồ chơi','Thể thao','Điện lạnh','Đồ văn phòng','Trang sức']
    for idx, t in enumerate(store_types,1):
        f.write(f"INSERT INTO LoaiCuaHang (ma_loai_cua_hang, ten_loai) VALUES ({idx}, '{esc(t)}');\n")
    f.write("\n")

    # --- 4. CuaHang ---
    f.write("-- 4. CuaHang\n")
    stores = []
    for i, store_acc in enumerate(store_accounts,1):
        name = f"{fake_company()}"
        address = fake_address()
        type_id = random.randint(1, len(store_types))
        store_id = int("225" + f"{random.randint(0,99999):05d}")
        stores.append({'id': store_id, 'account_id': store_acc['id']})
        f.write(f"INSERT INTO CuaHang (ma_cua_hang, ma_tai_khoan, ten_cua_hang, dia_chi, ma_loai_cua_hang) VALUES ({store_id}, {store_acc['id']}, '{esc(name)}', '{esc(address)}', {type_id});\n")
    f.write("\n")

    # --- 5. LoaiSanPham ---
    f.write("-- 5. LoaiSanPham\n")
    product_types = ['Áo len tăm','Áo khoác','Điện thoại','Quần jean','Giày thể thao','Đồng hồ','Túi xách','Mỹ phẩm','Sách','Đồ chơi trẻ em','Thể thao','Điện lạnh','Đồ gia dụng','Đồ văn phòng','Trang sức']
    for idx, t in enumerate(product_types,1):
        f.write(f"INSERT INTO LoaiSanPham (ma_loai_san_pham, ten_loai) VALUES ({idx}, '{esc(t)}');\n")
    f.write("\n")

    # --- 6. SanPham ---
    f.write("-- 6. SanPham\n")
    products = []
    for i in range(1, num_products+1):
        store = random.choice(stores)
        product_type_id = random.randint(1, len(product_types))
        # Tạo tên sản phẩm kiểu VN: "<Loại> <màu> <mã>"
        color = random.choice(["Đỏ","Xanh","Đen","Trắng","Hồng","Vàng","Xám","Xanh dương"])
        name = f"{product_types[product_type_id-1]} {color} #{random.randint(1,999)}"
        price = random.randint(100000,2000000)
        qty = random.randint(10,100)
        sku = f"PRD{i:03d}"
        products.append({'sku': sku, 'store_id': store['id'], 'price': price})
        f.write(f"INSERT INTO SanPham (ma_san_pham, ma_cua_hang, ma_loai_san_pham, ten_san_pham, gia, so_luong) VALUES ('{esc(sku)}', {store['id']}, {product_type_id}, '{esc(name)}', {price}, {qty});\n")
    f.write("\n")

    # --- 7. DonHang ---
    f.write("-- 7. DonHang\n")
    orders = []
    for i in range(1, num_orders+1):
        user_acc = random.choice(accounts)
        order_date = datetime(2025,1,1) + timedelta(days=random.randint(0,364))
        status = random.choice(['CHO_XAC_NHAN','DA_XAC_NHAN','DANG_GIAO','DA_GIAO','DA_HUY'])
        order_id = f"DH2025{i:05d}"
        orders.append({'id': order_id, 'user_id': user_acc['id'],'ngay_dat': str(order_date.date()),'status': status })
        f.write(f"INSERT INTO DonHang (ma_don_hang, ma_tai_khoan, ngay_dat, trang_thai) VALUES ('{esc(order_id)}', {user_acc['id']}, TO_DATE('{order_date.date()}','YYYY-MM-DD'), '{status}');\n")
    f.write("\n")

    # --- 8. ChiTietDonHang ---
    f.write("-- 8. ChiTietDonHang\n")

    order_details = []  # Lưu chi tiết đơn hàng để dùng cho phần đánh giá

    for i in range(1, num_order_details+1):
        order = random.choice(orders)
        product = random.choice(products)
        qty = random.randint(1,5)
        price = product['price']
        f.write(f"INSERT INTO ChiTietDonHang (ma_don_hang, ma_san_pham, so_luong, don_gia) VALUES ('{esc(order['id'])}', '{esc(product['sku'])}', {qty}, {price});\n")
        # 🔹 Thêm dòng này:
        order_details.append({
            "ma_don_hang": order["id"],
            "ma_san_pham": product["sku"],
            "so_luong": qty,
            "don_gia": price
    })
    f.write("\n")

    # --- 9. DonViVanChuyen ---
    f.write("-- 9. DonViVanChuyen\n")
    carrier_list = ['GHN','GHTK','VTP','VNPOST']
    carrier_names = ['Giao Hàng Nhanh','Giao Hàng Tiết Kiệm','Viettel Post','Bưu điện Việt Nam']
    for code,name in zip(carrier_list, carrier_names):
        f.write(f"INSERT INTO DonViVanChuyen (ma_don_vi_van_chuyen, ten_don_vi_van_chuyen) VALUES ('{esc(code)}', '{esc(name)}');\n")
    f.write("\n")

    # --- 10. NguoiVanChuyen ---
    f.write("-- 10. NguoiVanChuyen\n")
    for i in range(1, num_delivery_persons+1):
        carrier_code = random.choice(carrier_list)
        name = fake_vietnam_name()
        phone = fake_phone()
        plate = f"{random.randint(50,99)}A-{random.randint(10000,99999)}"
        f.write(f"INSERT INTO NguoiVanChuyen (ma_nguoi_van_chuyen, ma_don_vi_van_chuyen, ho_ten, sdt, bien_so_xe) VALUES ({1000+i}, '{esc(carrier_code)}', '{esc(name)}', '{phone}', '{plate}');\n")
    f.write("\n")

    # --- 11. ThongTinVanChuyen ---
    f.write("-- 11. ThongTinVanChuyen\n")
    for i in range(1, num_shipping+1):
        order = random.choice(orders)
        delivery_id = 1000 + random.randint(1,num_delivery_persons)
        order_date = datetime.strptime(order["ngay_dat"], "%Y-%m-%d")
        ship_date = order_date + timedelta(days=random.randint(0,10))  # giao từ 0–10 ngày sau khi đặt
        #status = random.choice(['CHO_GIAO','DANG_GIAO','DA_GIAO','HUY'])
        if random.random() < 0.35:
          status = 'DA_GIAO'
        else:
          status = random.choice(['CHO_XAC_NHAN','DA_XAC_NHAN','DANG_GIAO','DA_HUY'])
        ship_id = f"VC2025{i:05d}"
        address = fake_address()
        f.write(f"INSERT INTO ThongTinVanChuyen (ma_van_chuyen, ma_don_hang, ma_nguoi_van_chuyen, dia_chi_giao, ngay_giao_du_kien, trang_thai) VALUES ('{esc(ship_id)}', '{esc(order['id'])}', {delivery_id}, '{esc(address)}', TO_DATE('{ship_date.date()}','YYYY-MM-DD'), '{status}');\n")
    f.write("\n")


    # --- 12. ThongTinDanhGia ---
    # --- 12. ThongTinDanhGia ---
    f.write("-- 12. ThongTinDanhGia\n")

    # chỉ lấy đơn hàng đã giao
    orders_delivered = [o for o in orders if o["status"] == "DA_GIAO"]

    if not orders_delivered:
        print("⚠️ Không có đơn hàng nào đã giao thành công — không thể tạo đánh giá!")
    else:
        review_texts = [
            "Sản phẩm tốt, đóng gói kỹ",
            "Giao nhanh, đúng mô tả",
            "Chất lượng kém, cần nâng cấp",
            "Hài lòng",
            "Không như hình",
            "Sẽ ủng hộ tiếp"
        ]

        reviews = []
        for order in orders_delivered:
            # Lấy tất cả chi tiết sản phẩm trong đơn hàng này
            order_products = [ct for ct in order_details if ct["ma_don_hang"] == order["id"]]

            # Chọn ngẫu nhiên 1–3 sản phẩm trong đơn đó để đánh giá
            if order_products:
                selected_products = random.sample(order_products, k=min(len(order_products), random.randint(1, 3)))
                for product in selected_products:
                    review_text = random.choice(review_texts)
                    rating = random.randint(3, 5)
                    # ngày đánh giá sau khi giao hàng 3–15 ngày
                    review_date = datetime.strptime(order["ngay_dat"], "%Y-%m-%d") + timedelta(days=random.randint(3, 15))

                    f.write(
                        f"INSERT INTO ThongTinDanhGia (ma_san_pham, ma_tai_khoan, ma_don_hang, noi_dung, danh_gia, ngay_danh_gia) "
                        f"VALUES ('{product['ma_san_pham']}', {order['user_id']}, '{order['id']}', "
                        f"'{esc(review_text)}', {rating}, TO_DATE('{review_date.strftime('%Y-%m-%d')}', 'YYYY-MM-DD'));\n"
                    )

                    reviews.append({
                        "order_id": order["id"],
                        "user_id": order["user_id"],
                        "product_id": product["ma_san_pham"],
                        "rating": rating
                    })

        print(f"✅ Sinh {len(reviews)} dòng dữ liệu đánh giá.")
# kết thúc
print(f"✅ Đã tạo file SQL: {os.path.abspath(output_file)}")


✅ Sinh 82 dòng dữ liệu đánh giá.
✅ Đã tạo file SQL: /content/Qly_E_Commerce_Data.sql


In [8]:
from google.colab import files
files.download("Qly_E_Commerce_Data.sql")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>